In [1]:
import numpy as np
from scipy import signal, fftpack
from matplotlib import pyplot as plt
from scipy.io import loadmat
import cmath 
import pandas as pd

In [2]:
# ARCHIVO .mat
# Este archivo debe tener un registro de al menos 0.7 segundos después de la generación de la falla
mat = loadmat('Falla_AB_90%_001ohm.mat') # El archivo se selecciona desde la interfaz

# NUMERO DE MUESTRAS POR CICLO (PARA EL SUBMUESTREO)
fs_user_cycle = 32 # ESTA DATO VIENE DE LA INTERFAZ

In [3]:
def subsampling(time,data,fs,fk,fs_user_cycle):
    # time is the vector of time
    # data is the vector with the signal
    # fs_comtrade is the sample rate from the comtrade file
    # fk is the frequency of the system
    # fs_user_cycle is the sample rate given by user
    N1 = fs_user_cycle
    fs_cycle = fs/fk
    N=np.int(fs_cycle)
    N_tot = np.int(len(data)/fs_cycle)
    new_data = [0]
    new_time = [0]
    for i in np.arange(N_tot):
        xi=data[i*N:i*N+N]
        ti=time[i*N:i*N+N]
        new_data[i*N1:i*N1+N1] = signal.resample(xi, N1)
        new_time[i*N1:i*N1+N1] = np.linspace(ti[0], ti[-1], N1, endpoint=False)
        
    return (new_time,new_data)

In [4]:
def quantizer(data, quantizing_bits):
    # data is the vector with the signal
    # quantizing_bits is the number of bits for the converter
    # Quantizer - S&H and ADC
    quantizing_levels   = 2 ** quantizing_bits
    quantizing_step     = (np.max(data)-np.min(data)) / quantizing_levels
    quantizing_signal   = np.round (data / quantizing_step) * quantizing_step;
    
    return quantizing_signal

In [5]:
def DFT(time, data, fk, fs_user_cycle):
    # time is the vector of time
    # data is the vector with the signal
    # fk is the frequency of the system
    # fs_user_cycle is the sample rate given by user
    
    N=np.int(fs_user_cycle)
    N_tot = len(data)-N
    Xc = [0]*N_tot
    Xs = [0]*N_tot
    t = [0]*N_tot
    
    # Ciclo para el ventaneo
    for i in np.arange(N_tot):
        xi=data[i:i+N]
        t[i]=time[i]
        Xc_sum = 0
        Xs_sum = 0
        # Ciclo para el filtro coseno
        for k in np.arange(N):
            Xc_temp=xi[k]*np.cos(2*np.pi*k/(N))
            Xc_sum=Xc_sum+Xc_temp
            Xs_temp=xi[k]*np.sin(2*np.pi*k/(N))
            Xs_sum=Xs_sum+Xs_temp
            
        Xc[i]= 2/(N*np.sqrt(2))*Xc_sum
        Xs[i]= -2/(N*np.sqrt(2))* Xs_sum
        
    return t, Xc, Xs

In [6]:
# Reading time vector:
time = mat['t'] #Se conoce que todos los archivos vendrán con este vector incluido

N = len(time)
fs = np.int(np.ceil(len(time)/time[-1])-4)

print('Record has {} samples'.format(N))
print('Sampling rate is {} samples/sec.'.format(fs))


# Line frequency in Hz
fk = 60 # It can be set automatically


# Reading voltaje and currents
voltages_bus1 = np.empty(([len(time),3]))
currents_bus1 = np.empty(([len(time),3]))

col=0
for i in ['a','b','c']:
    voltages_bus1[:,col] = mat['vVt1'+i].ravel()
    currents_bus1[:,col] = mat['iCt1'+i].ravel()
    col=col+1

Record has 300001 samples
Sampling rate is 1000000 samples/sec.


In [7]:
# Subsampling voltaje and currents
N_tot = np.int(N*fk/fs)*fs_user_cycle
V_bus1_sub = np.empty(([N_tot,3]))
I_bus1_sub = np.empty(([N_tot,3]))
time_sub = np.empty(([N_tot,6]))
for i in np.arange(6):
    if i<3:
        time_sub[:,i], V_bus1_sub[:,i] = subsampling(time,voltages_bus1[:,i],fs,fk,fs_user_cycle)
    else:
        time_sub[:,i], I_bus1_sub[:,i-3] = subsampling(time,currents_bus1[:,i-3],fs,fk,fs_user_cycle)

In [8]:
quantizing_bits_V = 12 # Valor típico: 12 (Voltaje)
quantizing_bits_I = 16 # Valor típico: 16 (Corriente)
dig_V_bus1_sub = np.empty(([N_tot,3]))
dig_I_bus1_sub = np.empty(([N_tot,3]))
dig_V_bus2_sub = np.empty(([N_tot,3]))
dig_I_bus2_sub = np.empty(([N_tot,3]))
for i in np.arange(6):
    if i<3:
        dig_V_bus1_sub[:,i] = quantizer(V_bus1_sub[:,i], quantizing_bits_V)
    else:
        dig_I_bus1_sub[:,i-3] = quantizer(I_bus1_sub[:,i-3], quantizing_bits_I)

In [9]:
N_tot_DFT = N_tot-fs_user_cycle
Xc_bus1_V = np.empty(([N_tot_DFT,3]))
Xs_bus1_V = np.empty(([N_tot_DFT,3]))
Xc_bus1_I = np.empty(([N_tot_DFT,3]))
Xs_bus1_I = np.empty(([N_tot_DFT,3]))
X_bus1_V = np.empty(([N_tot_DFT,3]))
Y_bus1_V = np.empty(([N_tot_DFT,3]))
X_bus1_I = np.empty(([N_tot_DFT,3]))
Y_bus1_I = np.empty(([N_tot_DFT,3]))
t = np.empty(([N_tot_DFT,6]))

for i in np.arange(6):
    if i<3:
        #BUS1
        t[:,i], Xc_bus1_V[:,i], Xs_bus1_V[:,i] = DFT(time_sub[:,i], dig_V_bus1_sub[:,i], fk, fs_user_cycle)
        for j in range(len(Xc_bus1_V)):
            z_bus1_V = complex(Xc_bus1_V[j,i],Xs_bus1_V[j,i])
            X_bus1_V[j,i], Y_bus1_V[j,i] = cmath.polar(z_bus1_V)
            Y_bus1_V[j,i] = (Y_bus1_V[j,i])*180/np.pi
            if Y_bus1_V[j,i] < 0:
                Y_bus1_V[j,i] = 360 + Y_bus1_V[j,i]
    else:
        #BUS1
        t[:,i], Xc_bus1_I[:,i-3], Xs_bus1_I[:,i-3] = DFT(time_sub[:,i], dig_I_bus1_sub[:,i-3], fk, fs_user_cycle)
        for j in range(len(Xc_bus1_I)):
            z_bus1_I = complex(Xc_bus1_I[j,i-3],Xs_bus1_I[j,i-3])
            X_bus1_I[j,i-3], Y_bus1_I[j,i-3] = cmath.polar(z_bus1_I)
            Y_bus1_I[j,i-3] = (Y_bus1_I[j,i-3])*180/np.pi
            if Y_bus1_I[j,i-3] < 0:
                Y_bus1_I[j,i-3] = 360 + Y_bus1_I[j,i-3]

# Detección de falla

In [10]:
pos_tf1=0
for i in range(len(X_bus1_V)):
    V_nom = 115/np.sqrt(3)
    p = 0.8 #Criterio de comparación
    phA = X_bus1_V[i,0]
    phB = X_bus1_V[i,1]
    phC = X_bus1_V[i,2]
    if phA < V_nom*p or phB < V_nom*p or phC < V_nom*p:
        pos_tf1=i
        break

pos_tf = pos_tf1+fs_user_cycle*5 # Se esperan 4 ciclos para determinar 
print(time[pos_tf])

[0.000456]


In [11]:
ph_falla = np.empty(([1,6]), dtype=complex)
ph = ['A','B','C']
for i in range(len(ph)):
    ph_falla[0,i] = complex(Xc_bus1_V[pos_tf,i],Xs_bus1_V[pos_tf,i])
    ph_falla[0,i+3] = complex(Xc_bus1_I[pos_tf,i],Xs_bus1_I[pos_tf,i])

In [12]:
def seq_general(ph):
    # Se usan las fases en valores rectangulares
    phA = ph[0]
    phB = ph[1]
    phC = ph[2]
    a = -0.5 + (np.sqrt(3)/2)*1j
    s0=(phA+phB+phC)/3
    s1=(phA+a*phB+a*a*phC)/3
    s2=(phA+a*a*phB+a*phC)/3
    #s = [s0, s1, s2]
    return s0, s1, s2

def seq_pos(s1):
    a = -0.5 + (np.sqrt(3)/2)*1j
    phA1 = s1
    phB1 = a*a*s1
    phC1 = a*s1
    ph = [phA1, phB1, phC1]
    return ph
    
def seq_neg(s2):
    a = -0.5 + (np.sqrt(3)/2)*1j
    phA2 = s2
    phB2 = a*s2
    phC2 = a*a*s2
    ph = [phA2, phB2, phC2]
    return ph

In [13]:
# se crea las etiquetas para la comparación entre positiva y negativa
def pos_neg():
    etiqueta_pos_neg = [{'AG'},{'AB','ABG'},{'BG'},{'BC','BCG'},{'CG'},{'AC','ACG'}]

    grupos_pos_neg = [set()]*12
    fallas_pos_neg = np.empty(([12,2]))
    c = 0
    for i in range(len(fallas_pos_neg)):
        if (i%2)==0:
            grupos_pos_neg[i] = etiqueta_pos_neg[c] 
            if i==0:
                fallas_pos_neg[i,0:2] = [340,20]
            else:
                fallas_pos_neg[i,0:2] = [fallas_pos_neg[i-1,1],fallas_pos_neg[i-1,1]+40]
            c=c+1
        else:
            fallas_pos_neg[i,0:2] = [fallas_pos_neg[i-1,1],fallas_pos_neg[i-1,1]+20]
            if c<len(etiqueta_pos_neg):
                grupos_pos_neg[i] = etiqueta_pos_neg[c].union(grupos_pos_neg[i-1])
            else:
                grupos_pos_neg[i] = grupos_pos_neg[i-1].union(etiqueta_pos_neg[0])
    return grupos_pos_neg, fallas_pos_neg

In [14]:
# se crea las etiquetas para la comparación entre cero y negativa 
def cero_neg():
    etiqueta_cero_neg = [{'AG','BCG'},{'CG','ABG'},{'BG','ACG'}]

    grupos_cero_neg = [set()]*6
    fallas_cero_neg = np.empty(([6,2]))
    c = 0
    for i in range(len(fallas_cero_neg)):
        if (i%2)==0:
            grupos_cero_neg[i] = etiqueta_cero_neg[c] 
            if i==0:
                fallas_cero_neg[i,0:2] = [315,45]
            else:
                fallas_cero_neg[i,0:2] = [fallas_cero_neg[i-1,1],fallas_cero_neg[i-1,1]+90]
            c=c+1
        else:
            fallas_cero_neg[i,0:2] = [fallas_cero_neg[i-1,1],fallas_cero_neg[i-1,1]+30]
            if c<len(etiqueta_cero_neg):
                grupos_cero_neg[i] = etiqueta_cero_neg[c].union(grupos_cero_neg[i-1])
            else:
                grupos_cero_neg[i] = grupos_cero_neg[i-1].union(etiqueta_cero_neg[0])
    return grupos_cero_neg, fallas_cero_neg

In [15]:
def tipo_falla(s0,s1,s2):
    r0, th0 = cmath.polar(s0)
    r1, th1 = cmath.polar(s1)
    r2, th2 = cmath.polar(s2)
    
    grupos_pos_neg, fallas_pos_neg = pos_neg()
    grupos_cero_neg, fallas_cero_neg = cero_neg()
    if r2 < 0.25:
        type_f={'ABC'}
    else:
        if r0 > 0.02:
            grupo_mo = {'AG','BG','CG'}
            # Verificación cero-negativa
            th2_off = th2 - th0
            th0_off = th0 - th0
            des0 = (th2_off - th0_off)*180/np.pi
            if des0<0: des0 = 360+des0
            for i in range(len(fallas_cero_neg)):
                if des0>340 or des0<=20:
                    sel1=grupos_cero_neg[0]
                elif i!=0 and des0>fallas_cero_neg[i,0] and des0<=fallas_cero_neg[i,1]:
                    sel1=grupos_cero_neg[i]
            # Verificacion positiva-negativa
            th2_off = th2 - th1
            th1_off = th1 - th1
            des1 = (th2_off-th1_off)*180/np.pi
            if des1<0: des1 = 360+des1
            for i in range(len(fallas_pos_neg)):
                if des1>340 or des1<=20:
                    sel2=grupos_pos_neg[0]
                elif i!=0 and des1>fallas_pos_neg[i,0] and des1<=fallas_pos_neg[i,1]:
                    sel2=grupos_pos_neg[i]
            type_f = sel1.intersection(sel2)
            if type_f == set():
                type_f = sel1.intersection(grupo_mo)
        else:
            grupo_bi = {'AB','BC','AC'}
            th2_off = th2 - th1
            th1_off = th1 - th1
            des1 = (th2_off-th1_off)*180/np.pi
            if des1<0: des1 = 360+des1
            for i in range(len(fallas_pos_neg)):
                if des1>340 or des1<=20:
                    sel=(grupos_pos_neg[0])
                elif i!=0 and des1>fallas_pos_neg[i,0] and des1<=fallas_pos_neg[i,1]:
                    sel=(grupos_pos_neg[i])
            type_f = sel.intersection(grupo_bi)
    return type_f

In [16]:
# Detección del tipo de falla
ph_I = ph_falla[0,3:6]
s0_I,s1_I,s2_I = seq_general(ph_I)
print(abs(s2_I))
tipo = tipo_falla(s0_I,s1_I,s2_I)

3.0462946745940096


In [17]:
tipo

{'AB'}